In [1]:
import os, re, sys, json
import numpy as np, pandas as pd
import _config
reload (_config)
from _config import SHE3202_FQ_FILES, EXP_DESIGN_2901, OLIGO_LIBRARY, SHE3202_DIR, A_BC_OLIGOS_OUT, POSITIVE_CONTROLS_FILE




In [ ]:



#load oligo library from experimental design
oligos_lib = OLIGO_LIBRARY
oligos_lib["id"] = oligos_lib.index
positive_controls = pd.read_csv(POSITIVE_CONTROLS_FILE)

#load barcode-matched umis from dictionary reads
a_output = pd.read_csv(A_BC_OLIGOS_OUT)

#recover the list of bc_umi edges from the transcription data
#this takes a long time
print "loading transcripts"
#with open("../data/all_bcs2umis.json") as f:
#    umi_bcs_json = json.load(f)
    
    
print "getting oligos"
unique_oligos = a_output.oligo.unique()   t 
print "computing keys"
print "done with keys"
#this command takes a very long time...
unique_bcs =  [k for k,g in a_output.groupby("bc",) if len(g) == 1]
unique_bcs_set = set(unique_bcs)
unique_bc_oligos = a_output.loc[a_output.bc.isin(unique_bcs_set) ]

loading transcripts
getting oligos
computing keys
done with keys


In [ ]:
sxs = 0
fails = 0

all_shared_bcs = set()
exps = [e.strip() for e in """DLD1_WT_BR2
DLD1_WT_BR1
HCT116_WT_BR2
HCT116_WT_BR1
U2OS_WT_BR1
other
U2OS_WT_BR2
HCT116_Gemcitabine2uM_BR2
HCT116_Gemcitabine2uM_BR1""".splitlines()]
oligo_info = pd.DataFrame(index=[(exp,o) for o in unique_oligos for exp in exps])

oligos = unique_bc_oligos.groupby("oligo")
for exp in exps:
    print exp
    print "loading"
    with open("../data/all_{}_bcs2umis.json".format(exp)) as f:
        vals = json.load(f)
    

    
    print "done loading"
    cnt = 0 
    if exp =="other": continue
    for o, rows in oligos:
        obcs_set = set(rows.bc)
        shared_bcs = [b for b in obcs_set if b in vals]
        all_shared_bcs.update(shared_bcs)
        count_transcripts = len(set([t for b in shared_bcs for t in vals[b]]))
        oligo_info.at[(exp,o),"n_transcripts"] = count_transcripts
        oligo_info.at[(exp,o),"n_total_bcs"] = len(obcs_set)
        oligo_info.at[(exp,o),"n_observed_bcs"] = len(shared_bcs)
        
        cnt+= 1
        #if cnt > 1000: break
        #break
        if len(shared_bcs) > 0:
            oligo_info.at[(exp,o),"argmax_transcriptions_per_bcs"] = sorted(list(shared_bcs),key = lambda x: len(set(vals[x])))[-1]
            oligo_info.at[(exp,o),"max_transcriptions_per_bcs"] = len(set(vals[sorted(list(shared_bcs),key = lambda x: len(set(vals[x])))[-1]]))
            #sxs +=1
        else:
            pass
                #fails+=1
        #print fails, sxs
        
oligo_info["oligo"] = oligo_info.index.map(lambda x:x[1])
oligo_info["exp"] = oligo_info.index.map(lambda x:x[0])
oligo_info.to_csv("../data/0304_oligo_statistics.csv")


In [ ]:
oligos_lib = OLIGO_LIBRARY
oligos_lib["id"] = oligos_lib.index

In [20]:
len(oligo_info)

90135

# PLOTS #

In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt


In [ ]:
oligos_lib = OLIGO_LIBRARY
oligos_lib["id"] = oligos_lib.index
oligo_info2 = oligo_info.copy().join(oligos_lib, on ="oligo").dropna()

In [ ]:
positive_controls
positive_control_rows = oligo_info2.loc[oligo_info2.RefSeqID.isin(positive_controls.Name)]

Q: Given the list of positive control oligos which are known to be expressed, do we see these in the transcript data? Are they transcribed abundantly?

In [ ]:
oligo_info2.n_

In [ ]:

f, subs = plt.subplots(1,1)

s = subs
s.semilogy()
positive_control_info = oligo_info.loc[oligo_info.oligo.isin(positive_control_rows.id)]
bins = np.linspace(0,150,75)
s.hist(oligo_info2.n_transcripts, density=True, bins = bins, label="all oligos")
s.hist(positive_control_rows.n_transcripts, density=True, bins = bins, label="positive controls")
s.set_title("total transcript abundance distribution")
s.set_xlabel("transcript count")
s.set_ylabel("p(x)")
s.legend()



pass

In [ ]:
#test = oligo_info2
candid_regex = re.compile("candid_(\d+)")
neg_regex = re.compile("Neg_(\d+)")

mutant_regex = re.compile("mut(\d+)$")
oligo_info2["locus_ids"] = oligo_info2.RefSeqID.apply(lambda x: int(candid_regex.search(x).groups()[0]) if candid_regex.search(x) else neg_regex.search(x).groups()[0] )
oligo_info2["mutant_num"] = oligo_info2.RefSeqID.apply(lambda x: int(mutant_regex.search(x).groups()[0]) if mutant_regex.search(x) else 0 )
oligo_info2["is_cand"] = oligo_info2.RefSeqID.apply(lambda x: True if candid_regex.search(x) else False )
oligo_info2["is_neg"] = oligo_info2.RefSeqID.apply(lambda x: True if neg_regex.search(x) else False )


In [ ]:
oligo_info2.mutant_num.unique()

In [ ]:
transcript_groups = oligo_info2.groupby("mutant_num").n_transcripts

h = plt.hist([e[1].values for e in transcript_groups], bins = np.linspace(0,20,10),density=True,label = [k for k,v in transcript_groups])
plt.legend()
#plt.hist([list(v for v in grp) for grp in transcript_groups], bins = np.linspace(0,20), density = True)
#plt.legend()

#TABLE 1#
Enrichment counts for each oligo in each cell conditioned, in a table

In [ ]:
enrichments = pd.DataFrame()
enrichments_vs_all = pd.DataFrame()
observations_vs_all = pd.DataFrame()
#enrichments.index =pd.Series( oligo_info2.oligo.unique())
for k,g in oligo_info2.groupby("exp"):
    #print  g.apply(lambda x:x.n_transcripts / x.n_observed_bcs)
    enrichments[k] = pd.Series(g.apply(lambda x:x.n_transcripts / x.n_observed_bcs, axis = 1).values,index = g.apply(lambda x:x.id,axis=1))
    enrichments_vs_all[k] = pd.Series(g.apply(lambda x:x.n_transcripts / x.n_total_bcs, axis = 1).values,index = g.apply(lambda x:x.id,axis=1))
    observations_vs_all[k] = pd.Series(g.apply(lambda x:x.n_observed_bcs / x.n_total_bcs, axis = 1).values,index = g.apply(lambda x:x.id,axis=1))
    #enrichments[k+"_vs_bcs"] = g.apply(lambda x:x.n_transcripts / x.n_observed_bcs, axis = 1)


In [ ]:
f,subs = plt.subplots(3,1)
f.set_size_inches(8,12)
subs[0].hist(enrichments.values.transpose(), density = True)

In [ ]:
print "hi"

In [ ]:
g.apply(lambda x:x.n_transcripts / x.n_observed_bcs, axis = 1).values

In [ ]:
pd.Series(g.apply(lambda x:x.n_transcripts / x.n_observed_bcs, axis = 1).values,index = g.apply(lambda x:x.id,axis=1))